**Importing libraries, functions and dependencies**

In [18]:
# To work with directories
import os 
        
# To plot curves and graphs
import matplotlib.pyplot as plt

# To display images
from IPython.display import display
from IPython.display import Image as _Imgdis

import math # For mathematical operations and inbuilt functions
import numpy as np # Linear algebra

# To open, save, display, resize images and convert it to grayscale
import cv2 
from PIL import Image

# To split the data into test and train samples
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


# To build the model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout, Input
from tensorflow.keras import layers

# For model summary
from tensorflow.keras.utils import plot_model

# To generate the model report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

path = './'
for files in os.listdir(path):
     os.remove(files)

import warnings
warnings.filterwarnings('ignore')

**Create a function to display images**

In [19]:
Img_size = 265

In [20]:
def show(path):
    display(_Imgdis(filename=path, width=265, height=265))  

**Importing the 'data' and 'labels' arrays**

In [21]:
data = np.load("../input/arrays/data/data.npy")
labels = np.load("../input/arrays/labels/labels.npy")
print(data.shape,labels.shape)

**Splitting the data into test and test samples**

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, 
                                                    random_state=100)

In [23]:
# Implementing the to_categorical() function
y_train = to_categorical(y_train, 32)
y_test = to_categorical(y_test, 32)

# Rescaling the pixel values
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

print(X_train.shape, y_train.shape)

**Instantiate the VGG16 model**

In [24]:
base_model = VGG16(input_shape = (Img_size, Img_size,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
import tensorflow as tf

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(1000, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(32, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy',metrics = ['acc'])

In [25]:
model.summary()

In [26]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

**Running the model**

In [27]:
history = model.fit(X_train, y_train, batch_size=40, validation_data=(X_test, y_test),steps_per_epoch = 100, 
                    epochs = 10)

**Testing the model on test samples**

In [28]:
y_test_pred = model.predict(X_test)

**Plotting the learning curve**

In [29]:
# Retrieve accuracy history on training and validation data
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve loss history on training and validation data
loss = history.history['loss']
val_loss = history.history['val_loss']

# Obtain the number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, label='training accuracy')
plt.plot(epochs, val_acc, label='validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

# Plot training and validation loss per epoch
plt.figure()
plt.plot(epochs, loss, label='training loss')
plt.plot(epochs, val_loss, label='validation loss')
plt.title('Training and validation loss')
plt.legend()

**Model metrics**

In [30]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted, pos_label='positive',
                                          average='micro'))
    print("Recall = " ,recall_score(y_actual, y_predicted, pos_label='positive',
                                           average='micro'))
    print("F1 Score = " ,f1_score(y_actual, y_predicted,pos_label='positive',
                                           average='micro'))

generate_model_report(np.argmax(y_test, axis=1), np.argmax(y_test_pred, axis=1))

**Defining a prediction function**

In [31]:
def prediction(image):
    image = Image.open(image)
    image=image.resize((Img_size,Img_size))
    image = np.reshape(image, (1,Img_size,Img_size,3))
    image = image/255.0
    result = model.predict(image)
    result = np.argmax(result, axis=1)[0]
    return result

print(prediction('../input/plantleafidentification/Leaves/1006.jpg'))
show('../input/plantleafidentification/Leaves/1006.jpg')

**Testing the model on the validation dataset**

In [32]:
path2='../input/plantleafidentification-2'

tp = 0
for image in os.listdir(path2):
    image_path = path2+"/"+image
    image = image.replace(".jpg","")
    t = int(float(image))
    p = prediction(image_path)
    if t==p :
        tp = tp + 1
        show(image_path) # To show the images whose label is predicted correctly

print('validation accuracy is:', tp/0.52)

**Saving the model**

In [33]:
model.save('model.h5')

# To save the model in the output file in kaggle:
# model.save('./hero_predictor.h5')
